In [ ]:
'''
Here is the extra experiments file we test the VAE, we did not show it in the paper because of the space
Generally speaking, we test the EC2VAE and SketchVAE in the disentanglement of rhythm and pitch.
We change the rhythm but maintain the pitch/change the pitch but maintain the rhythm, to see the changes of z_pitch and z_rhythm.
Please ***RUN THE CODE WITH OPTIONS*** in the cell.
'''
import numpy as np
import os
import shutil
import random
import torch
import sklearn.utils
from torch.distributions import kl_divergence,Normal
from torch.nn import functional as F
from torch import optim

# These three data is achived by the code in the below cell, 
# about how we use the oringinal data to shift pitches and rhythms
data_path = [
    "data/exp_vae_independent.npy",
    "data/exp_vae_independent_pitch.npy",
    "data/exp_vae_independent_rhythm.npy"
]
batch_size = 16

In [ ]:
# DON'T RUN THIS CODE
# Because data has already there
def check_rest(x):
    for i in x:
        if i == 129:
            return False
    return True

def shift_pitch(x, p):
    new_x = []
    for i in range(len(x)):
        if x[i] < 100:
            new_x.append(x[i] + p)
        else:
            new_x.append(x[i])
    return new_x

def shift_rhythm(x):
    x_p = []
    x_new = [] 
    j = 0
    for i in range(len(x)):
        if x[i] < 128:
            x_p.append(x[i])
    pos = 0
    if x[0] < 128:
        x_new.append(x_p[pos])
        pos += 1
    for j in x[::-1]:
        if j < 128:
            x_new.append(x_p[pos])
            pos += 1
        else:
            x_new.append(128)
        if pos >= len(x_p):
            break
    while pos < len(x_p):
        x_new.append(x_p[pos])
        pos += 1
    while len(x_new) < len(x):
        x_new.append(128)
    return x_new

train_x = np.load(ori_data_path,allow_pickle=True)
print(train_x.shape)
print(train_x[10].shape)
new_x  = []
new_rx = []
new_px = []
for i in train_x:
    for j in i:
        if check_rest(j):
            rj = shift_rhythm(j)
            pj = shift_pitch(j,6)
            if rhythm_acc(j,rj) < 1:
                new_x.append(j)
                new_rx.append(rj)
                new_px.append(pj)
new_x = np.array(new_x)
new_rx = np.array(new_rx)
new_px = np.array(new_px)
new_x = np.split(new_x[:len(new_x) // 16 * 16], len(new_x) // 16)
new_rx = np.split(new_rx[:len(new_rx) // 16 * 16], len(new_rx) // 16)
new_px = np.split(new_px[:len(new_px) // 16 * 16], len(new_px) // 16)
new_x = np.array(new_x)
new_rx = np.array(new_rx)
new_px = np.array(new_px)
np.save("vae_data/exp_vae_independent.npy", new_x)
np.save("vae_data/exp_vae_independent_pitch.npy",new_px)
np.save("vae_data/exp_vae_independent_rhythm.npy",new_rx)

In [ ]:
# accuracy and  extratcion
def rhythm_acc(a,b):
    s = len(a)
    num_pitch = np.sum(a >= 128)
    acc = np.sum(np.logical_and(a == b,  a >= 128)) / num_pitch
    return acc

# note extraction
hold_state = 128
rest_state = 129
def extract_note(x, pad_token = 128):
    d = []
    for i in x:
        if i < 128:
            d.append(i)
    ori_d = len(d)
    d.extend([pad_token] * (len(x) - len(d)))
    return np.array(d), ori_d

def extract_rhythm(x, hold_token = 2, rest_token = 3):
    d = []
    for i in x:
        if i < 128:
             d.append(1)
        elif i == hold_state:
             d.append(hold_token)
        else:
             d.append(rest_token)
    return np.array(d)

In [ ]:
# Option-1 SketchVAE
from SketchVAE.sketchvae import SketchVAE
from torch.utils.data import Dataset, DataLoader, TensorDataset
###############################
# initial parameters

save_path = "model_backup/"
lr = 1e-4
decay = 0.9999
hidden_dims = 1024
zp_dims = 128
zr_dims = 128
vae_beta = 0.1
input_dims = 130
pitch_dims = 129
rhythm_dims = 3
seq_len = 6 * 4
beat_num = 4
tick_num = 6
save_period = 1
##############################


In [ ]:
# Option-2 EC2VAE
from ec2vae.model_dis import RECVAE
###############################
# ec_vae
hidden_dims = 2048
z1_dims = 128
z2_dims = 128
vae_beta = 0.1
input_dims = 130
rhythm_dims = 3
seq_len = 6 * 4
##############################

In [ ]:
# input data
data_x = np.load(data_path[0], allow_pickle = True)
data_px = np.load(data_path[1], allow_pickle = True)
data_rx = np.load(data_path[2], allow_pickle = True)
print(data_x[0][0])
print(data_px[0][0])
# dl = DataLoader(train = train_x, validate = train_x, test = train_x)

In [ ]:
# Option-1 SketchVAE
# input data
def processed_data_tensor(data):
    print("processed data:")
    gd = [] 
    px = []
    rx = []
    len_x = []
    nrx = []
    for d in data:
        gd.append([list(dd[0]) for dd in d])
        px.append([list(dd[1]) for dd in d])
        rx.append([list(dd[2]) for dd in d])
        len_x.append([dd[3] for dd in d])
        if len(gd[-1][-1]) != 24:
            gd[-1][-1].extend([128] * (seq_len - len(gd[-1][-1])))
            px[-1][-1].extend([128] * (seq_len - len(px[-1][-1])))
            rx[-1][-1].extend([2] * (seq_len - len(rx[-1][-1])))
    for i,d in enumerate(len_x):
        for j,dd in enumerate(d):
            if len_x[i][j] == 0:
                gd[i][j][0] = 60
                px[i][j][0] = 60
                rx[i][j][0] = 1
                len_x[i][j] = 1
                print("find all 0")
            
    gd = np.array(gd)
    px = np.array(px)
    rx = np.array(rx)
    len_x = np.array(len_x)
    for d in rx:
        nnrx = []
        for dd in d:
            temp = np.zeros((seq_len, rhythm_dims))
            lins = np.arange(0, len(dd))
            temp[lins, dd - 1] = 1
            nnrx.append(temp)
        nrx.append(nnrx)
    nrx = np.array(nrx)
    gd = torch.from_numpy(gd).long()
    px = torch.from_numpy(px).long()
    rx = torch.from_numpy(rx).float()
    len_x = torch.from_numpy(len_x).long()
    nrx = torch.from_numpy(nrx).float()
    print("processed finish!")
    print(gd.size(),px.size(),rx.size(),len_x.size(),nrx.size())
    return TensorDataset(px, rx, len_x, nrx, gd)

split_size = 24
new_data = []
for i,d in enumerate(data_x):
    ds = d
    data = []
    for sd in ds:
        q,k = extract_note(sd)
        s = extract_rhythm(sd)
        data.append([sd,q,s,k])
    new_data.append(data)
dis_data_x = np.array(new_data)

new_data = []
for i,d in enumerate(data_px):
    ds = d
    data = []
    for sd in ds:
        q,k = extract_note(sd)
        s = extract_rhythm(sd)
        data.append([sd,q,s,k])
    new_data.append(data)
dis_data_px = np.array(new_data)

new_data = []
for i,d in enumerate(data_rx):
    ds = d
    data = []
    for sd in ds:
        q,k = extract_note(sd)
        s = extract_rhythm(sd)
        data.append([sd,q,s,k])
    new_data.append(data)
dis_data_rx = np.array(new_data)


temp = DataLoader(
    dataset = processed_data_tensor(dis_data_x),
    batch_size = 1, 
    shuffle = False, 
    num_workers = 8, 
    pin_memory = True, 
    drop_last = True
)
dis_data_x = []
for i,d in enumerate(temp):
    dis_data_x.append(d)
print(len(dis_data_x))

temp = DataLoader(
    dataset = processed_data_tensor(dis_data_px),
    batch_size = 1, 
    shuffle = False, 
    num_workers = 8, 
    pin_memory = True, 
    drop_last = True
)
dis_data_px = []
for i,d in enumerate(temp):
    dis_data_px.append(d)
print(len(dis_data_px))

temp = DataLoader(
    dataset = processed_data_tensor(dis_data_rx),
    batch_size = 1, 
    shuffle = False, 
    num_workers = 8, 
    pin_memory = True, 
    drop_last = True
)
dis_data_rx = []
for i,d in enumerate(temp):
    dis_data_rx.append(d)
print(len(dis_data_rx))

In [ ]:
# Option-1 SketchVAE
# import model
model = SketchVAE(input_dims, pitch_dims, rhythm_dims, hidden_dims, zp_dims, zr_dims, seq_len, beat_num, tick_num, 4000)
dic = torch.load("model_backup/sketchvae-param.pt")

for name in list(dic.keys()):
    dic[name.replace('module.', '')] = dic.pop(name)
model.load_state_dict(dic)

if torch.cuda.is_available():
    print('Using: ', torch.cuda.get_device_name(torch.cuda.current_device()))
    model.cuda()
else:
    print('Using: CPU')



In [ ]:
# Option-1 SketchVAE
zs = []
zps = []
zrs = []
# p(z)

model.eval()
device = torch.device(torch.cuda.current_device())

for i in range(len(dis_data_x)):
    v_px, v_rx, v_len_x, v_nrx, v_gd = dis_data_x[i]
    v_px = v_px[0]
    v_rx = v_rx[0]
    v_len_x = v_len_x[0]
    v_nrx = v_nrx[0]
    v_gd = v_gd[0]
    v_px = v_px.to(device = device,non_blocking = True)
    v_len_x = v_len_x.to(device = device,non_blocking = True)
    v_nrx = v_nrx.to(device = device,non_blocking = True)
    v_gd = v_gd.to(device = device,non_blocking = True)
    _, v_p_dis, v_r_dis, _ = model(v_px, v_nrx, v_len_x, v_gd)
    zp = v_p_dis.mean
    zr = v_r_dis.mean
    z = torch.cat((zp,zr), -1)
    z = z.cpu().detach().numpy()
    zs.append(z)
    if i % 100 == 0:
        print("processed : %d" %i)
zs = np.array(zs)

for i in range(len(dis_data_px)):
    v_px, v_rx, v_len_x, v_nrx, v_gd = dis_data_px[i]
    v_px = v_px[0]
    v_rx = v_rx[0]
    v_len_x = v_len_x[0]
    v_nrx = v_nrx[0]
    v_gd = v_gd[0]
    v_px = v_px.to(device = device,non_blocking = True)
    v_len_x = v_len_x.to(device = device,non_blocking = True)
    v_nrx = v_nrx.to(device = device,non_blocking = True)
    v_gd = v_gd.to(device = device,non_blocking = True)
    _, v_p_dis, v_r_dis, _ = model(v_px, v_nrx, v_len_x, v_gd)
    zp = v_p_dis.mean
    zr = v_r_dis.mean
    z = torch.cat((zp,zr), -1)
    z = z.cpu().detach().numpy()
    zps.append(z)
    if i % 100 == 0:
        print("processed : %d" %i)
zps = np.array(zps)

for i in range(len(dis_data_rx)):
    v_px, v_rx, v_len_x, v_nrx, v_gd = dis_data_rx[i]
    v_px = v_px[0]
    v_rx = v_rx[0]
    v_len_x = v_len_x[0]
    v_nrx = v_nrx[0]
    v_gd = v_gd[0]
    v_px = v_px.to(device = device,non_blocking = True)
    v_len_x = v_len_x.to(device = device,non_blocking = True)
    v_nrx = v_nrx.to(device = device,non_blocking = True)
    v_gd = v_gd.to(device = device,non_blocking = True)
    _, v_p_dis, v_r_dis, _ = model(v_px, v_nrx, v_len_x, v_gd)
    zp = v_p_dis.mean
    zr = v_r_dis.mean
    z = torch.cat((zp,zr), -1)
    z = z.cpu().detach().numpy()
    zrs.append(z)
    if i % 100 == 0:
        print("processed : %d" %i)
zrs = np.array(zrs)

In [ ]:
# Option-2 EC2VAE
# load EC_VAE model

model = RECVAE(input_dims, hidden_dims, rhythm_dims, z1_dims,z2_dims,seq_len, 3000)

dic = torch.load("model_backup/ec2vae-param.pt")
for name in list(dic.keys()):
    dic[name.replace('module.', '')] = dic.pop(name)
model.load_state_dict(dic)

if torch.cuda.is_available():
    print('Using: ', torch.cuda.get_device_name(torch.cuda.current_device()))
    model.cuda()
else:
    print('Using: CPU')
model.eval()

In [ ]:
# Option-2 EC2VAE
# EC_VAE decode
def convert_onehot(batch_data):
    vecs_out = []
    for d in batch_data:
        midi_vec = np.zeros((len(d),130))
        k = np.arange(len(d))
        midi_vec[k,d] = 1
        vecs_out.append(midi_vec)
    return np.array(vecs_out)

i = 0
for d in data_x:
    raw_x = d
    if len(raw_x[-1]) != seq_len:
        raw_x = raw_x[:-1]
    raw_x = np.array(raw_x)
    x = convert_onehot(raw_x)
    x = torch.from_numpy(x).float()
    if torch.cuda.is_available():
        x = x.cuda()
    _,_,rdis1,rdis2 = model(x)
    z1 = rdis1.mean
    z2 = rdis2.mean
    z = torch.cat((z1,z2),-1)
    z = z.cpu().detach().numpy()
    zs.append(z)
    i += 1
    if i % 100 == 0:
        print("processed : %d" %i)
zs = np.array(zs)

i = 0
for d in data_px:
    raw_x = d
    if len(raw_x[-1]) != seq_len:
        raw_x = raw_x[:-1]
    raw_x = np.array(raw_x)
    x = convert_onehot(raw_x)
    x = torch.from_numpy(x).float()
    if torch.cuda.is_available():
        x = x.cuda()
    _,_,rdis1,rdis2 = model(x)
    z1 = rdis1.mean
    z2 = rdis2.mean
    z = torch.cat((z1,z2),-1)
    z = z.cpu().detach().numpy()
    zps.append(z)
    i += 1
    if i % 100 == 0:
        print("processed : %d" %i)
zps = np.array(zps)

i = 0
for d in data_rx:
    raw_x = d
    if len(raw_x[-1]) != seq_len:
        raw_x = raw_x[:-1]
    raw_x = np.array(raw_x)
    x = convert_onehot(raw_x)
    x = torch.from_numpy(x).float()
    if torch.cuda.is_available():
        x = x.cuda()
    _,_,rdis1,rdis2 = model(x)
    z1 = rdis1.mean
    z2 = rdis2.mean
    z = torch.cat((z1,z2),-1)
    z = z.cpu().detach().numpy()
    zrs.append(z)
    i += 1
    if i % 100 == 0:
        print("processed : %d" %i)
zrs = np.array(zrs)

In [ ]:
# Option-2 EC2VAE
domi = zs.shape[0] * zs.shape[1]
print("change rhythm zp:", np.sum(np.absolute(zs[:, :, :128] - zrs[:,:, :128])) / domi )
print("change rhythm zr:",  np.sum(np.absolute(zs[:, :, 128:] - zrs[:,:, 128:])) / domi)
print("change pitch zp:", np.sum(np.absolute(zs[:, :, :128] - zps[:,:, :128])) / domi)
print("change pitch zr:", np.sum(np.absolute(zs[:, :, 128:] - zps[:,:, 128:])) / domi)

In [ ]:
# Option-1 SketchVAE
domi = zs.shape[0] * zs.shape[1]
print("change rhythm zp:", np.sum(np.absolute(zs[:, :, :128] - zrs[:,:, :128])) / domi )
print("change rhythm zr:",  np.sum(np.absolute(zs[:, :, 128:] - zrs[:,:, 128:])) / domi)
print("change pitch zp:", np.sum(np.absolute(zs[:, :, :128] - zps[:,:, :128])) / domi)
print("change pitch zr:", np.sum(np.absolute(zs[:, :, 128:] - zps[:,:, 128:])) / domi)